In [4]:
import json
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import os

In [5]:
data_path = '../data/DeckLists/'
deck_files = [f for f in listdir(data_path) if isfile(join(data_path, f)) and 'Identifier' not in f]

In [8]:
#dict -->  keys are the cards w/ a list 
#dataframe columns = deckname as str, card name as str, count as int, sideboard as bool
deck = {'deckname':[], 'cardname':[], 'card_count':[], 'sideboard':[]}


for file_name in deck_files:
    file = open(data_path+file_name, 'r')
    sideboard = False
    for line in file:
        #print(line)
        items = line.split(" ")
        if '\n' in items:
            sideboard = True
        elif items[0][:4] == 'Deck' or items[0][:9] == 'Sideboard' or items[0][:9] == 'Companion':
            pass
        else:
            try:
                deck['deckname'].append(file_name)
                deck['cardname'].append(" ".join(items[1:]).replace('\n',''))
                deck['card_count'].append(int(items[0]))
                deck['sideboard'].append(sideboard)
            except (Exception, ValueError) as error:
                print(f'Unable to process: {error}')
                print(file)
                

parsed_df = pd.DataFrame(deck)

In [9]:
parsed_df

,deckname,cardname,card_count,sideboard
0,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),Thrashing Brontodon,2,False
1,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),Trail of Crumbs,3,False
2,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),Kazandu Mammoth,4,False
3,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),"Vorinclex, Monstrous Raider",2,False
4,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),Lovestruck Beast,4,False
...,...,...,...,...
1139,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(4),"Klothys, God of Destiny",2,True
1140,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(4),"Vivien, Monsters' Advocate",2,True
1141,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(4),Mystical Dispute,3,True
1142,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(4),Ox of Agonas,2,True


In [10]:
import psycopg2 as psy
from psycopg2 import sql
from io import StringIO
import sqlalchemy
from sqlalchemy import create_engine

def connect(connection_details):  
    """accepts connection details, establishes connection and returns it"""
    conn = None
    try:
        print('Connecting to PostgreSQL database...')
        conn = psy.connect(**connection_details)
    except (Exception, psy.DatabaseError) as error:
        print(f'Unable to connect to the database: {error}')
        sys.exit(1)
    print('Connection successful')
    return conn

    
con_details = {"dbname" : 'pay_to_play', 
               "user" : os.environ['PGSQL_P_USER'], 
               "password" : os.environ['PGSQL_P_PWD'], 
               "host" : 'localhost'}    

conn = connect(con_details)

Connecting to PostgreSQL database...
Connection successful


In [11]:
# def add_uuid(deck_df, conn, uuid_field, sql_lookup_field, df_lookup_field, table):
#     uuid = sql.Identifier(uuid_field)
#     query = sql.SQL("SELECT {fields} FROM {tbl}").format(
#         fields = sql.SQL(', ').join([sql.Identifier(uuid_field), sql.Identifier(sql_lookup_field)]),
#         tbl = sql.Identifier(table))
#     name_id_df = pd.read_sql(query, conn).groupby(sql_lookup_field).max()
#     return deck_df.merge(name_id_df, how = 'left', left_on=df_lookup_field, right_on=sql_lookup_field)
        
def add_uuid_to_deck(deck_df, conn, df_name_field):
    query = sql.SQL("""
        WITH newestsetdate AS (
            SELECT name, MAX(setreleasedate) as max_date
            FROM core
            JOIN setdetails ON "setCode" = "setcode"
            GROUP BY name
            ),
            newestset AS (
            SELECT name, setcode
            FROM newestsetdate
            JOIN setdetails ON newestsetdate.max_date = setdetails.setreleasedate
            )

            SELECT core.uuid, newestset.name
            FROM core
            INNER JOIN newestset ON newestset.name = core.name AND newestset.setcode = core."setCode";""")
    name_id_df = pd.read_sql(query, conn).groupby("name").max() 
    return deck_df.merge(name_id_df, how = 'left', left_on=df_name_field, right_on="name")

    

In [12]:
uuid_deck = add_uuid_to_deck(parsed_df, conn, "cardname")

In [13]:
uuid_deck.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1144 entries, 0 to 1143
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   deckname    1144 non-null   object
 1   cardname    1144 non-null   object
 2   card_count  1144 non-null   int64 
 3   sideboard   1144 non-null   bool  
 4   uuid        1010 non-null   object
dtypes: bool(1), int64(1), object(3)
memory usage: 45.8+ KB


In [14]:
uuid_deck

,deckname,cardname,card_count,sideboard,uuid
0,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),Thrashing Brontodon,2,False,523c83b0-ee94-5f0a-9ad9-f7efb5aad95e
1,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),Trail of Crumbs,3,False,2e5c5bd2-ed4a-556f-a920-10ee0ddcb6cc
2,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),Kazandu Mammoth,4,False,NaN
3,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),"Vorinclex, Monstrous Raider",2,False,f85a6483-b319-54a8-ac93-3dfe883fd166
4,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),Lovestruck Beast,4,False,NaN
...,...,...,...,...,...
1139,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(4),"Klothys, God of Destiny",2,True,bed702eb-e2ed-5989-9341-3b1198980180
1140,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(4),"Vivien, Monsters' Advocate",2,True,5e52bd2a-f453-58d1-ab8a-9d42295f445e
1141,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(4),Mystical Dispute,3,True,5651bd95-cb41-5266-8654-93707f0c4f9a
1142,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(4),Ox of Agonas,2,True,82cd6def-fef5-5e08-8af8-934f1be45237


In [55]:
def add_new_card_data(conn_details, card_inf_df, table, conn):
    engine_path = "postgresql+psycopg2://" + conn_details['user'] + ":" + conn_details['password']  
    engine_path +='@localhost:5432/' +conn_details['dbname']
    engine = create_engine(engine_path)
    try:
        card_inf_df.to_sql(table, engine, index=False, if_exists='append')
    except (Exception, psy.DatabaseError) as error:
        print(f'Error: {error}')
        conn.rollback()
        return 1
    conn.commit()
    print(f'Successful updating of {table}')
    



In [56]:

missing_uuids = uuid_deck[uuid_deck['uuid'] != uuid_deck['uuid']].drop('uuid', axis=1)
missing_uuids.head()
deck_load = uuid_deck[uuid_deck['uuid'] == uuid_deck['uuid']].drop('cardname', axis = 1)
deck_load.head()

,deckname,card_count,sideboard,uuid
0,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),2,False,523c83b0-ee94-5f0a-9ad9-f7efb5aad95e
1,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),3,False,2e5c5bd2-ed4a-556f-a920-10ee0ddcb6cc
3,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),2,False,f85a6483-b319-54a8-ac93-3dfe883fd166
5,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),4,False,a12d890a-1619-5545-b2f0-3897d6c61c8b
6,Platinum-Mythic-Rank-Player-Traditional-(Bo3)(7),4,False,3ef6edae-e899-521c-a731-3aeeff8b154e


In [57]:
add_new_card_data(con_details, deck_load , 'decklists', conn)

Successful updating of decklists


In [58]:
add_new_card_data(con_details, missing_uuids, 'nan_uuid_decklists', conn)

Successful updating of nan_uuid_decklists


In [ ]:
# 45 unmatched.  373 matched.  Need to now build a multi-field text-scan/match for name to address these.
